# Country-level vegetation carbon stock

The purpose of this notebook is to serve as interactive documentation for the Vegetation Carbon Stock aggregation script. It is structured in a way that the functions are used immediately after they are declared, allowing the user to perform the country-level aggregation process step-by-step. All the functions defined in this notebook are identical to their Python script counterparts. 

The script begins by importing all the packages needed to perform the country-level aggregation of vegetation carbon stock:

In [4]:
import os
import geopandas as gpd
import rasterio
import rasterio.mask
from affine import Affine
import numpy as np
import pandas as pd
import math
import platform

## Input data
The script requires two inputs:
- the global carbon stock rasters for each year,
- the country polygons shapefile.

Later in the script, carbon stock values will be aggregated inside each country polygon. 

In [ ]:
vcs_rasters_directory = r"\\akif.internal\public\veg_c_storage_rawdata" # This is the directory containing carbon stock data. 
country_polygons_file = r"\\akif.internal\public\z_resources\im-wb\2015_gaul_dataset_mod_2015_gaul_dataset_global_countries_1.shp" # This is the shapefile storing the country polygons.

Given the path to the directory containing the carbon stock rasters for each year, the function `get_raster_data` iterates over all the files in the directory and stores the paths inside a list. Later, the script will iterate over this list to load the carbon stock data and perform the country-level aggregation year-by-year. 

In [ ]:
def get_raster_data(path):
    """
    get_raster_data creates a list containing the path to each of the files in a given directory.
    :param path: the path to the directory.
    :return: the list containing the path to each file in the directory.
    """
    file_list = []
    for file in os.listdir(path):
        # Iterate over all the files in the specified directory.
        if ".tif" in file:
            # Process the file if it has a .tif format.
            # Build the path according the OS running the script.
            if platform.system() is "Windows":
                address = os.path.join(path, file).replace("/","\\")
            else:
                address = os.path.join(path, file).replace("\\","/")
                

            if address not in file_list:
                # Add the file address to the list if it had not been added before.
                file_list.append(address)
        else:
            pass
    return file_list

vcs_raster_list = get_raster_data(vcs_rasters_directory)

The countries' polygons are the same for each year of the analysis, thus a single shapefile is loaded as a GeoDataFrame with the `load_country_polygons` function:

In [ ]:
def load_country_polygons(file):
    """
    load_country_polygons loads the shapefile storing the countries' polygons in a GeoDataFrame.
    :param file: the path to the shapefile.
    :return: a GeoDataFrame with the data on the countries' polygons.
    """
    # Build the path according the OS running the script.
    if platform.system() is "Windows":
        file = file.replace("/","\\")
    else:
        file = file.replace("\\","/")
        
    gdf = gpd.read_file(file)
    return gdf

country_polygons = load_country_polygons(country_polygons_file)

## Aggregation process

This section describes the aggregation process. The aggregation for every year is done inside the function `carbon_stock_aggregation`, however a number of subprocesses used there are defined in separate functions for clarity. In the following lines, these functions are declared and their purpose explained one-by-one.  

### Calculating pixel real-area accounting for pixel latitude

The global data produced by ARIES is in tonnes of vegetation carbon per hectare, thus the calculation of the total carbon stock per country in tonnes requires the multiplication of each pixel's value by its area. Although in the planar projection all pixels are identical squares, their real area depends on the pixel's latitude, because of the ellipsoid shape of the Earth. The function `area_of_pixel` calculates the area of a pixel given its side length and the latitude of its center:


In [ ]:
def area_of_pixel(pixel_size, center_lat):
    """
    area_of_pixel calculates the area, in hectares, of a wgs84 square raster tile. 
                  This function is adapted from https://gis.stackexchange.com/a/288034.
    :param pixel_size: the length of the pixel side in degrees.              
    :param center_lat: the latitude of the center of the pixel. This value +/- half the `pixel-size` must not exceed 90/-90 degrees latitude or an invalid area will be calculated.
    :return: the area of a square pixel of side length `pixel_size` whose center is at latitude `center_lat` in hectares.
    """
    a = 6378137  # meters
    b = 6356752.3142  # meters
    e = math.sqrt(1 - (b/a)**2)
    area_list = []
    for f in [center_lat+pixel_size/2, center_lat-pixel_size/2]:
        zm = 1 - e*math.sin(math.radians(f))
        zp = 1 + e*math.sin(math.radians(f))
        area_list.append(
            math.pi * b**2 * (
                math.log(zp/zm) / (2*e) +
                math.sin(math.radians(f)) / (zp*zm)))
    return (pixel_size / 360. * (area_list[0] - area_list[1])) * np.power(10.0,-4) 

# TODO: Ruben puedes completar y corregir estos ejemplos?
area_of_pixel_equator      =  area_of_pixel(0.1,0.0)
area_of pixel_tropic       =  area_of_pixel(0.1,23.5)
area_of_pixel_polar_circle =  area_of_pixel(0.1,66.33)

### Calculating the total carbon stock in a given region 

Knowing the area of each pixel as a function of its latitude, the calculation of the total carbon stock in a region requires to simply sum for each pixel its value in carbon stock per hectares times its area. The function `get_total_carbon_stock` performs this operation with the raster `out_image`: 

In [ ]:
def get_total_carbon_stock(out_image, out_transform, pixel_size, width_0, height_0, width_1, height_1):
    """
    get_total_carbon_stock calculates the total carbon stock from a raster with with data on the carbon stock per hectare.
    
    :param out_image: is the baseline raster layer, in the context of this script the vegetation carbon stock raster.
    :param out_transform: the Affine containing the transformation matrix with latitude and longitude values, resolution, etc.
    :param : TODO: Completar el resto de parametros.
    :param pixel_size: is the side length in degrees of each square raster tile.
    :return: the total carbon stock extracted from the raster.
    """
    # Create a matrix of coordinates based on tile number.
    cols, rows = np.meshgrid(np.arange(width_0, width_1), np.arange(height_0, height_1))
    
    # Transform the tile number coordinates to real coordinates and extract only latitude information. 
    ys = rasterio.transform.xy(out_transform, rows, cols)[1] # [0] is xs
    latitudes = np.array(ys) # Cast the list of arrays to a 2D array for computational convenience.

    # Iterate over the latitudes matrix, calculate the area of each tile, and store it in the real_raster_areas array.
    real_raster_areas = np.empty(np.shape(latitudes))
    for i, latitude_array in enumerate(latitudes):
        for j, latitude in enumerate(latitude_array):
            real_raster_areas[i,j] = area_of_pixel(pixel_size, latitude)

    # Calculate the total carbon stock in each tile: tonnes/hectare * hectares = tonnes.    
    total_carbon_stock_array = real_raster_areas * out_image[0,height_0:height_1,width_0:width_1] #I don't think np.transpose() is necesary

    # Sum all the carbon stock values in the country treating NaNs as 0.0. 
    total_carbon_stock = np.nansum(total_carbon_stock_array) 

    return total_carbon_stock

# TODO: Ruben se te ocurre como poner algun ejemplo con esto?

For most of the countries, `get_total_carbon_stock` is called at whole the country-level, however when the `out_image` raster of the country is too large in size calling the function results in a buffer overflow.

### Splitting large countries in different sections to avoid buffer overflows

Some large countries, such as Russia, or the United States, as well as some archipelagic countries require to load in memory rasters that are too large in size, potentially causing memory errors. The purpose of the function `carbon_stock_raster_tiling` is to avoid memory errors by splitting problematic countries' carbon stock rasters in smaller square sections when the raster size exceeds 3Gb. Inside the function, the vegetation carbon stock is calculated for each section separately and then summed to obtain the total value for the country:

In [ ]:
def carbon_stock_raster_tiling(out_image, out_transform, pixel_size, width, height):
    """
    raster_tiling calculates the carbon stock in large countries by splitting the country in sections and then aggregating each section's stock.
    
    :param out_image: the masked raster layer, in the context of this script the vegetation carbon stock raster.
    :param out_transform: the Affine containing the transformation matrix with latitude and longitude values, resolution...
    :param pixel_size: the side length in degrees of each square raster tile.
    :param width: the width of the masked layer.
    :param height: the height of the masked layer.   
    :return: the value of the total carbon stock for the country.
    """
    tilesize = 1000
    total_acumulated_carbon_stock = 0

    for i in range(0, width, tilesize): #tilesize marks from where to where in width
        for j in range(0, height, tilesize):
            #this is for the edge parts, so we don't get nodata from the borders
            w0 = i #start of the array
            w_plus = min(i+tilesize, width) - i #addition value
            w1 = w0 + w_plus #end of the array
            h0 = j #start of the array
            h_plus = min(j+tilesize, height) - j #addition value
            h1 = h0 + h_plus #end of the array

            total_carbon_stock = get_total_carbon_stock(out_image, out_transform, pixel_size, w0, h0, w1, h1)

            total_acumulated_carbon_stock += total_carbon_stock

    return total_acumulated_carbon_stock

# TODO: Ruben te imaginas algun ejemplo aqui?

### Aggregating carbon stock for every country and every year

Finally, the functions declared above are used to perform the aggregation process for every country and every year. The function `carbon_stock_aggregation` simply iterates over each year's global carbon stock raster file and each country and masks the global raster file with the polygon corresponding to the country. Then it calculates the aggregated carbon stocks calling `get_total_carbon_stock` or `raster_tiling` when the masked rasters are larger than 3Gb. The results are progressively stored in a DataFrame that is exported to CSV format after every year. When all the years were correctly processed, the function returns the complete DataFrame with the carbon stock per country for every year of the analysis.

In [ ]:
def carbon_stock_aggregation(raster_files_list, country_polygons):
    """
    carbon_stock_aggregation aggregates vegetation carbon stock data in Tonnes per Hectare and with a resolution of 300m at the country level. 
                             The result of the aggregation is the total vegetation carbon stock in Tonnes for each country. Naturally, the 
                             dependence of raster tile area on the latitude is taken into account. The function iterates over the carbon stock 
                             rasters corresponding to different years.
    
    :param raster_files_list: a list containing the addresses of all the raster files that store the vegetation carbon stock data for each year.
    :param country_polygons: a GeoDataFrame storing the polygons corresponding to each country for the entire world.
    :return: a DataFrame storing the aggregated vegetation carbon stocks at the country level for each year.
    """
    
    # Final DataFrame will store the aggregated carbon stocks for each country and each year. 
    aggregated_carbon_stock_df = pd.DataFrame([])
    
    for file in raster_files_list[:]: # [10:]
        # Iterate over all the raster files' addresses and extract the year from the address. 
        filename_length = 24 # This is the number of characters in the raster file name if the convention "vcs_YYYY_global_300m.tif" is followed.
        start = len(file) - filename_length
        year_string_start = file.find("vcs_",start)
        file_year = str( file[ year_string_start + 4 : year_string_start + 8] )
        
        print("We are working with the file {} from the year {}".format(file, file_year))

        aggregated_carbon_stock_list = [] # This list will store the results from the aggregation. 

        with rasterio.open(file) as raster_file: # Load the raster file.

            gt = raster_file.transform # Get all the raster properties on a list.
            pixel_size = gt[0] # X size is stored in position 0, Y size is stored in position 4.

            for row_index, row in country_polygons.iterrows(): # gdf.loc[0:1].iterrows():
                # Iterate over the country polygons to progressively calculate the total carbon stock in each one of them.
                
                geo_row = gpd.GeoSeries(row['geometry']) # This is the country's polygon geometry.

                # Masks the raster over the current country. The masking requires two outputs:
                # out_image: the array of the masked image. [z, y, x]
                # out_transform: the Affine containing the transformation matrix with lat / long values, resolution...
                out_image, out_transform = rasterio.mask.mask(raster_file, geo_row, crop=True) 
                
                # Obtain the number of tiles in both directions.
                height = out_image.shape[1]
                width  = out_image.shape[2]

                #check the size of the raster image
                if out_image.nbytes > (3* 10**9):
                    print("the country {} exceeds 3Gb of memory, we will split the array in tiles of 1000. Current size is GB: {} ".format(row["ADM0_NAME"], (out_image.nbytes) / np.power(10.0,9)))

                    total_carbon_stock = carbon_stock_raster_tiling(out_image, out_transform, pixel_size, width, height)

                else:
                    # Create a global raster where each pixel's value corresponds to its true area in hectares.
                    total_carbon_stock = get_total_carbon_stock(out_image, out_transform, pixel_size, 0, 0, width, height) 
                
                # Add the aggregated stock to the list.
                aggregated_carbon_stock_list.append(total_carbon_stock)  

                print("the country {} is finished".format(row["ADM0_NAME"]))
                
        print("Finished calculating {} year raster".format(file_year))
    
        # Transform the list to a DataFrame using the year as header.
        aggregated_carbon_stock = pd.DataFrame(aggregated_carbon_stock_list, columns = [file_year]) 

        # Merge this year's carbon stocks to the final, multi-year DataFrame.
        aggregated_carbon_stock_df = pd.merge(aggregated_carbon_stock_df, aggregated_carbon_stock, how='outer', left_index = True, right_index=True)

        #export the carbon stock year as a backup 
        aggregated_carbon_stock.to_csv("carbon_stock_{}.csv".format(file_year))

    return aggregated_carbon_stock_df

# TODO: ejemplo estaría bien también. Algo que pueda correr rapidito en un laptop personal.

### Output data

After the aggregation process is finished, the generated DataFrame is exported to a file in CSV format.

In [ ]:
def export_to_csv(country_polygons, aggregated_carbon_stocks):   
    """
    export_to_csv creates a "total_carbon_test.csv" file in the current working directory that contains the total vegetation carbon stock for each country.
    
    :param country_polygons: a GeoDataFrame storing the polygons corresponding to each country for the entire world.
    :param aggregated_carbon_stocks: a DataFrame storing the aggregated carbon stock values to be associated with each country.
    """
    # Create a DataFrame based on the country border GeoDataFrame and dropping unnecessary information to keep only: the polygons' Id, country codes, and administrative names.
    df_final = pd.DataFrame(country_polygons.drop(columns='geometry'))
    df_final = df_final.drop(["STATUS", "DISP_AREA", "ADM0_CODE", "STR0_YEAR", "EXP0_YEAR", "Shape_Leng", "ISO3166_1_", "ISO3166__1", "Shape_Le_1", "Shape_Area"], axis = 1)

    # Join the depurated country DataFrame with the aggregated vegetation carbon stocks to associate each country with its total stock.  
    df_final = df_final.join(aggregated_carbon_stocks)
        
    # Export the result to the current working directory.
    df_final.to_csv("total_carbon.csv")

## Running the whole script

In the Python script the analysis is performed after declaring every function and with the following commands: 

In [ ]:
# Loading the Input data.
print("Loading data.")
vcs_rasters_list = get_raster_data(vcs_rasters_directory) 
country_polygons = load_country_polygons(country_polygons_file) 
print("Data was loaded succesfully.")

# Performing the country-level aggregation for every year.
print("Starting aggregation process.")
vcs_aggregated   = carbon_stock_aggregation(vcs_rasters_list, country_polygons) 
print("Aggregation of vegetation carbon stocks at the country level finished.")

# Exporting final DataFrame to CSV.
print("Exporting the results.")
export_to_csv(country_polygons, vcs_aggregated) 
print("Total vegetation carbon stocks at the country level succesfully exported.")